In [1]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

In [2]:
candidate_labels = ["animals", "animals activity", "water", "scenery", "weather", "human activity", "language", "plants"]

In [3]:
import pandas as pd
words = pd.read_csv("words.csv")
docs = list(words["Definitions"])

In [4]:
results = pd.DataFrame([], columns = candidate_labels)
for doc in docs:
    scores = classifier(doc, candidate_labels, )["scores"]
    labels = classifier(doc, candidate_labels, )["labels"]
    results = pd.concat([results, pd.DataFrame(scores).transpose().set_axis(labels=labels, axis= 1)], axis = 0)

In [5]:
results

,animals,animals activity,water,scenery,weather,human activity,language,plants
0,0.038399,0.073731,0.098784,0.039909,0.046750,0.086811,0.586388,0.029228
0,0.016083,0.047442,0.028530,0.030171,0.027242,0.634231,0.201780,0.014522
0,0.033712,0.073668,0.036069,0.026115,0.037753,0.706134,0.058586,0.027963
0,0.009064,0.020933,0.020706,0.012812,0.007356,0.874492,0.047878,0.006759
0,0.327679,0.638166,0.006772,0.004991,0.003473,0.008011,0.007967,0.002941
...,...,...,...,...,...,...,...,...
0,0.028799,0.118041,0.026910,0.070485,0.061894,0.544067,0.087510,0.062294
0,0.041889,0.132756,0.137194,0.098814,0.061169,0.287516,0.182547,0.058115
0,0.044979,0.131243,0.116675,0.100754,0.079019,0.300945,0.164314,0.062071
0,0.019359,0.039077,0.043358,0.048516,0.028053,0.718047,0.083571,0.020019


In [6]:
results.to_csv("initial_results_zero_shot.csv")

In [7]:
len(docs)

3804